# Lecture 23


### Underdetermined linear systems

[notes](https://drive.google.com/file/d/1sW1sAYc7K0Q0fHTeEOLmZ9ZZVclO6EmI/view?usp=sharing)

## A cautionary tale for MATLAB's guessing

In [26]:
A = randn(10,6);
b = randn(10,1);
x = zeros(6);

In [38]:
y = A'*b - A'*(A*x);
v = A*y;
h = (y'*y)/(v'*v);

## A cautionary tale for the normal equations

To minimize

$$ \|A \vec y - \vec b \|_2^2 + \lambda \|\vec y \|_2^2, \quad \lambda = 10^{-16}$$

In [55]:
A = randn(4,10); b = randn(4,1);
An = [ 1e-8*eye(10); A];
bn = [ zeros(10,1); b];

In [51]:
(An'*An)\(An'*bn)


ans =

  -0.374162336527059
  -0.725925472907776
  -0.304243712255422
  -0.377124915445571
   0.101047611754736
   0.449085092446448
  -0.247460507782943
   0.673961644951171
  -0.259743496234919
   0.618693041065650



New normal equations with $\gamma = 10^{-16}$

In [52]:
format long; rank(A'*A)


ans =

     4



In [53]:
size(An'*An)


ans =

    10    10



In [56]:
rank(An'*An)


ans =

     4



In [58]:
norm(An'*An - A'*A)


ans =

     9.661765613766217e-16



In [59]:
(A'*A)\(A'*b)



ans =

   0.475398978202305
   0.382061257980468
   1.016800178663588
  -0.682207279721434
  -0.459538058525725
  -0.208791290440113
   0.028311765113459
  -0.868950972021673
  -0.225048019101538
  -0.092362791906641



## The solution here is to avoid the normal equations and use the QR method!!!!

## An application

Suppose your rock company produces

* 20 tons of road base (RB) per day 
* 10 tons of sand (S) per day
* 10 tons of gravel (G) per day

You have 4 types of equipment and you are wondering how to increase production by approximately 50% while attempting to buy/sell as little equipment as possible.

Equipment:

As a simplification, the estimated production of each type of equipment per hour is caputed in a vector
$$ \begin{pmatrix} \text{RG per day} \\ \text{S per day} \\ \text{G per day} \end{pmatrix}.$$

* Truck:  1 hour produces $$ \begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix} $$
* Shovel A:  1 hour produces $$ \begin{pmatrix} 0 \\ 2 \\ 1 \end{pmatrix} $$
* Mixer:  1 hour produces $$ \begin{pmatrix} 1 \\ -1/2 \\ -1/2 \end{pmatrix} $$
* Shovel B:  1 hour produces $$ \begin{pmatrix} 0 \\ 1 \\ 1 \end{pmatrix} $$

We will also assume that every type of equipment costs the same amount.

The current operations can be captured as a vector $\vec x$ such that
$$ \begin{pmatrix} 0 & 0 & 1 & 0 \\ 0 & 2 & -1/2 & 1 \\ 1 & 1 & -1/2 & 1 \end{pmatrix} \vec x = \begin{pmatrix} 20 \\ 10 \\10\end{pmatrix} $$
We are looking for a new vector $\vec x_{\text{new}}$ such that
$$ \begin{pmatrix} 0 & 0 & 1 & 0 \\ 0 & 2 & -1/2 & 1 \\ 1 & 1 & -1/2 & 1 \end{pmatrix} \vec x_{\text{new}} \approx \begin{pmatrix} 30 \\ 15 \\15\end{pmatrix} $$

To minimized the buying and selling of equipment, we want to keep $\vec x - \vec x_{\text{new}}$ small.  Note that
$$ \begin{pmatrix} 0 & 0 & 1 & 0 \\ 0 & 2 & -1/2 & 1 \\ 1 & 1 & -1/2 & 1 \end{pmatrix} (\vec x_{\text{new}} - \vec x) \approx \begin{pmatrix} 10 \\ 5 \\ 5\end{pmatrix}$$

So we look to minimize
$$ \| A \vec y - \vec b \|_2^2 + \|\vec y\|_2^2$$
where
$$ A = \begin{pmatrix} 0 & 0 & 1 & 0 \\ 0 & 2 & -1/2 & 1 \\ 1 & 1 & -1/2 & 1 \end{pmatrix}, \quad \vec b= \begin{pmatrix} 10 \\ 5 \\ 5\end{pmatrix},$$
to find $d\vec x = x_{\text{new}} - \vec x$.

In [ ]:
A = [0 0 1 0; 0 2 -.5 1; 1 1 -.5 1]; b = [10; 5; 5];

In [ ]:
An = [ eye(4) ; A]; bn = [zeros(4,1);b];  % eye(4) could change depending on relative
% costs of each type of machine

In [ ]:
dx = (An'*An)\(An'*bn)

In [ ]:
[~,R] = House([An,bn])

In [ ]:
dx = Backsub(R(1:4,:))

In [ ]:
sqrt(norm(A*dx-b)^2 + norm(dx)^2)

The change in production is given by:

In [ ]:
A*dx

This does not seem like a big enough increase in production of road base.  So we change the optimization:

In [ ]:
An = [ eye(4) ; A]; bn = [zeros(4,1);b];
[~,R] = House([An,bn]);
dx = Backsub(R(1:4,:))
extra_production = A*dx
needed_equipment = sum(dx)

In [ ]:
An = [ .5*eye(4) ; A]; bn = [zeros(4,1);b];
[~,R] = House([An,bn]);
dx = Backsub(R(1:4,:))
extra_production = A*dx
needed_equipment = sum(dx)

This can help give us a clear view of the trade-offs of different solutions